In [3]:
import re
import numpy as np
import pandas as pd 
from konlpy.tag import Okt
import pyLDAvis
import pyLDAvis.gensim
from gensim import corpora 
from gensim import models
import gensim

In [4]:
def trans(rowdata,kingname):
    rmhdata = []
    for num1 in range(len(rowdata)):
        for num in range(len(rowdata[num1])):
            rmhdata.append(re.sub('[(一-龥)]','',rowdata[num1][num]))
    wdata =[]
    for i in range(len(rmhdata)):
        wdata.append(re.sub('-|○|/|。|=|\n','',rmhdata[i]))
    
    sdata =[]
    for i in range(len(wdata)):
        if wdata[i] != '':
            sdata.append(wdata[i])
    ssdata =[]
    for i in range(len(sdata)):
        if re.search('[가-힣]+',sdata[i]):
            ssdata.append(sdata[i])
    data =[]
    for i in range(len(ssdata)):
        if kingname not in ssdata[i]:
           data.append(ssdata[i])

    return data

In [5]:
# 키워드 : 백성

In [6]:
data =[]
for i in range(0,1):
    txt1 = open('태조실록.txt',encoding='utf-8').readlines()
    date = []
    keywordtxt =[]
    for num,i in enumerate(txt1):
        if '태조실록' in i:
            if ('년'and '월' and '일') in i:
                date.append(num)

    for k in range(len(date)-1):
        count =0
        for j in range(date[k],date[k+1]):
            if '백성' in txt1[j]:
                count +=1
        if count >0:
            keywordtxt.append(txt1[date[k]:date[k+1]])
    data.append(trans(keywordtxt,'태조실록'))

In [7]:
# 시리즈 만들기
df=pd.Series(keywordtxt)

In [8]:
#한글만 남기기
df=df.map(lambda x : re.sub('[^ ㄱ-ㅣ가-힣]+','',str(x)))

In [9]:
df=df.map(lambda x : x.replace("태조실록"," "))
df=df.map(lambda x : x.replace("태조"," "))
df=df.map(lambda x : x.replace("번째기사"," "))
df=df.map(lambda x : x.replace("홍무"," "))

In [10]:
len(df)

152

In [11]:
okt = Okt()

In [12]:
#형태소로 분리
stopword = ['전하','임금','사람','여러','나라','로써','또한','지금','어찌','모두','우리'
           ]
text=[]
for i in range(0,len(df)):
    t=okt.nouns(str(df[i]))
    temp =[]
    for j in t:
        if j not in stopword:
            if len(j)>1:
                temp.append(j)
    text.append(temp)

In [13]:

dictionary = corpora.Dictionary(text)
corpus = [dictionary.doc2bow(t) for t in text]


In [14]:
 # 문서-단어 행렬 만들어주는 함수.
def build_doc_term_mat(documents):

    dictionary = corpora.Dictionary(documents)
    corpus = [dictionary.doc2bow(document) for document in documents]
        
    return corpus, dictionary


In [15]:
NUM_TOPICS = 20

NUM_TOPIC_WORDS = 10

def print_topic_words(model):
    print("\nPrinting topic words.\n")
    
    for topic_id in range(model.num_topics):
        topic_word_probs = model.show_topic(topic_id, NUM_TOPIC_WORDS)
        print("Topic ID: {}".format(topic_id))
        
        for topic_word, prob in topic_word_probs:
            print("\t{}\t{}".format(topic_word, prob))
            
        print("\n")

corpus, dictionary = build_doc_term_mat(text)
model = gensim.models.ldamodel.LdaModel(corpus, num_topics=NUM_TOPICS, id2word=dictionary, alpha="auto", eta="auto")
print_topic_words(model)


Printing topic words.

Topic ID: 0
	생각	0.007582643069326878
	백성	0.005497490055859089
	군사	0.004775363020598888
	가지	0.0036584939807653427
	조정	0.003408276243135333
	함부로	0.0033784923143684864
	감히	0.0033253561705350876
	마음	0.0032858424820005894
	반드시	0.0032589926850050688
	하늘	0.002931643510237336


Topic ID: 1
	참으로	0.009816746227443218
	도읍	0.008524883538484573
	백성	0.007494485471397638
	고치	0.0071396371349692345
	한우	0.005296667106449604
	하늘	0.004516472574323416
	마음	0.004319463856518269
	양반	0.004112769849598408
	명당	0.00409333361312747
	생각	0.003965055104345083


Topic ID: 2
	백성	0.011125284247100353
	생각	0.007161945570260286
	태백성	0.004985891282558441
	신하	0.0043686553835868835
	도읍	0.0043065776117146015
	조선	0.004251168109476566
	왕위	0.0036988640204072
	감히	0.003639881731942296
	마음	0.0034128588158637285
	하늘	0.0034007877111434937


Topic ID: 3
	백성	0.011222964152693748
	궁궐	0.006054329685866833
	종묘	0.00584630249068141
	제사	0.0052075316198170185
	공사	0.004529412370175123
	생각	0.004311969969421625
	군사	0.00413

In [16]:
# 주제 :  백성
pyLDAvis.enable_notebook()
data = pyLDAvis.gensim.prepare(model, corpus, dictionary)
data 


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
8     -0.040305 -0.004409       1        1  28.028414
9     -0.011455  0.025409       2        1  10.313085
16    -0.021210  0.030126       3        1   9.530662
7      0.054276  0.041867       4        1   7.160122
2      0.016210 -0.015979       5        1   6.406779
19    -0.039981 -0.023925       6        1   6.247386
14    -0.024026  0.007862       7        1   5.170728
5     -0.000105  0.005524       8        1   4.966341
15    -0.012030  0.015769       9        1   2.953194
13    -0.007672 -0.025514      10        1   2.787859
18    -0.017593  0.005903      11        1   2.670614
4     -0.047299  0.010111      12        1   2.272473
12     0.024824  0.014554      13        1   2.132886
10     0.028674 -0.006378      14        1   2.086090
11     0.022447 -0.032584      15        1   2.065302
1      0.033903 -0.054452      16        1   1.592881
3     -0.002801 -0.008637      17        1   1.505099
6      0.038489  0.022975      18        1   0.965870
0      0.006765  0.003731      19        1   0.703007
17    -0.001111 -0.011953      20        1   0.441218, topic_info=     Term        Freq       Total Category  logprob  loglift
36     백성  318.000000  318.000000  Default  30.0000  30.0000
49     생각  121.000000  121.000000  Default  29.0000  29.0000
12     군사   87.000000   87.000000  Default  28.0000  28.0000
992    도읍   49.000000   49.000000  Default  27.0000  27.0000
910    하늘   92.000000   92.000000  Default  26.0000  26.0000
...   ...         ...         ...      ...      ...      ...
1054   이익    0.181323   21.413912  Topic20  -6.2498   0.6519
147    명령    0.206264   45.967892  Topic20  -6.1209   0.0168
34    반드시    0.212621   60.669804  Topic20  -6.0905  -0.2303
111    황제    0.208956   62.580597  Topic20  -6.1079  -0.2787
219    지방    0.194437   35.664253  Topic20  -6.1799   0.2116

[1435 rows x 6 columns], token_table=      Topic      Freq Term
term                      
2111      1  0.688839   가금
1514      1  0.289560   가뭄
1514      2  0.096520   가뭄
1514      4  0.096520   가뭄
1514      5  0.096520   가뭄
...     ...       ...  ...
3808      1  0.318502   흠차
3808      3  0.159251   흠차
3808      4  0.159251   흠차
2050      5  0.438405   힐난
2050      6  0.438405   힐난

[2992 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[9, 10, 17, 8, 3, 20, 15, 6, 16, 14, 19, 5, 13, 11, 12, 2, 4, 7, 1, 18])

In [17]:
#주제 : 군사

In [18]:
data =[]
for i in range(0,1):
    txt1 = open('태조실록.txt',encoding='utf-8').readlines()
    date = []
    keywordtxt =[]
    for num,i in enumerate(txt1):
        if '태조실록' in i:
            if ('년'and '월' and '일') in i:
                date.append(num)

    for k in range(len(date)-1):
        count =0
        for j in range(date[k],date[k+1]):
            if '군사' in txt1[j]:
                count +=1
        if count >0:
            keywordtxt.append(txt1[date[k]:date[k+1]])
    data.append(trans(keywordtxt,'태조실록'))

In [19]:
df=pd.Series(keywordtxt)
df=df.map(lambda x : re.sub('[^ ㄱ-ㅣ가-힣]+','',str(x)))
df=df.map(lambda x : x.replace("태조실록"," "))
df=df.map(lambda x : x.replace("태조"," "))
df=df.map(lambda x : x.replace("번째기사"," "))
df=df.map(lambda x : x.replace("홍무"," "))

In [20]:
len(df)

112

In [21]:
stopword = ['전하','임금','사람','여러','나라','로써','또한','지금','어찌','우리','서로'
           ]
text=[]
for i in range(0,len(df)):
    t=okt.nouns(str(df[i]))
    temp =[]
    for j in t:
        if j not in stopword:
            if len(j)>1:
                temp.append(j)
    text.append(temp)

In [22]:
dictionary = corpora.Dictionary(text)
corpus = [dictionary.doc2bow(t) for t in text]

In [23]:
NUM_TOPICS = 20

NUM_TOPIC_WORDS = 10

def print_topic_words(model):
    print("\nPrinting topic words.\n")
    
    for topic_id in range(model.num_topics):
        topic_word_probs = model.show_topic(topic_id, NUM_TOPIC_WORDS)
        print("Topic ID: {}".format(topic_id))
        
        for topic_word, prob in topic_word_probs:
            print("\t{}\t{}".format(topic_word, prob))
            
        print("\n")

corpus, dictionary = build_doc_term_mat(text)
model = gensim.models.ldamodel.LdaModel(corpus, num_topics=NUM_TOPICS, id2word=dictionary, alpha="auto", eta="auto")
print_topic_words(model)


Printing topic words.

Topic ID: 0
	모두	0.012467185035347939
	군사	0.009140643291175365
	백성	0.008381029590964317
	하늘	0.00537429004907608
	비록	0.004176322370767593
	관찰사	0.004166296683251858
	마음	0.003818465629592538
	고려	0.0037815303076058626
	정도전	0.0037356195971369743
	처음	0.003640070091933012


Topic ID: 1
	군사	0.0095111895352602
	모두	0.008820164017379284
	백성	0.0076255230233073235
	태종	0.006683384533971548
	명운	0.0038225403986871243
	황제	0.003773595904931426
	옛날	0.003688251366838813
	하늘	0.003561032470315695
	반드시	0.0034345663152635098
	생각	0.0031928783282637596


Topic ID: 2
	군사	0.010743789374828339
	모두	0.009427794255316257
	정안군	0.00722876563668251
	백성	0.006539891008287668
	마음	0.004828972741961479
	하소	0.003653791267424822
	하늘	0.003553127171471715
	수령	0.003449950134381652
	반드시	0.003374109510332346
	고려	0.0031768304761499166


Topic ID: 3
	백성	0.010947160422801971
	모두	0.010332435369491577
	군사	0.006252930033951998
	아들	0.003795088268816471
	비록	0.003545835381373763
	생각	0.003525251755490899
	제사	0.00345469

In [27]:
# 주제 :  군사
pyLDAvis.enable_notebook()
data = pyLDAvis.gensim.prepare(model, corpus, dictionary)
data 

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.001474  0.012037       1        1  12.732885
10     0.001288  0.024576       2        1  12.319973
7     -0.038200 -0.075207       3        1  10.370948
15     0.022634  0.000622       4        1   9.885950
12     0.016240 -0.020177       5        1   9.012307
19     0.036818 -0.022884       6        1   8.145066
13     0.012560  0.003782       7        1   6.426342
17     0.006178  0.013043       8        1   4.557155
14    -0.017789  0.027511       9        1   3.608588
4     -0.029457  0.005843      10        1   3.411882
5      0.015368  0.021338      11        1   3.311051
1      0.017984 -0.015826      12        1   3.221832
2      0.006515  0.021970      13        1   3.064954
9      0.008330 -0.012834      14        1   2.968461
16     0.004776 -0.026857      15        1   1.967199
6     -0.003805  0.006643      16        1   1.943240
8      0.022865  0.005752      17        1   1.627851
11    -0.007708  0.006363      18        1   0.679548
18    -0.075826  0.016343      19        1   0.500818
3     -0.000244  0.007962      20        1   0.243958, topic_info=     Term        Freq       Total Category  logprob  loglift
1      군사  216.000000  216.000000  Default  30.0000  30.0000
42     백성  149.000000  149.000000  Default  29.0000  29.0000
33     모두  155.000000  155.000000  Default  28.0000  28.0000
2308  정안군   77.000000   77.000000  Default  27.0000  27.0000
422    고치   37.000000   37.000000  Default  26.0000  26.0000
...   ...         ...         ...      ...      ...      ...
360    먼저    0.085982   35.168072  Topic20  -6.2936   0.0022
128    계책    0.081971   26.380980  Topic20  -6.3413   0.2419
1011   왕씨    0.083528   37.467686  Topic20  -6.3225  -0.0901
111    하니    0.085388   56.757458  Topic20  -6.3005  -0.4834
807    다만    0.082367   36.255417  Topic20  -6.3365  -0.0712

[1365 rows x 6 columns], token_table=      Topic      Freq Term
term                      
3631      2  0.164771   가선
3631      3  0.164771   가선
3631      7  0.329542   가선
3631     14  0.164771   가선
1216     11  0.282306   가속
...     ...       ...  ...
4370      9  0.140781   흥무
4370     12  0.140781   흥무
4371      4  0.547160  흥무왕
1631      5  0.665084   힐난
3888      5  0.644836   힘껏

[2853 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 11, 8, 16, 13, 20, 14, 18, 15, 5, 6, 2, 3, 10, 17, 7, 9, 12, 19, 4])

In [28]:
#주제 : 제사

In [29]:
data =[]
for i in range(0,1):
    txt1 = open('태조실록.txt',encoding='utf-8').readlines()
    date = []
    keywordtxt =[]
    for num,i in enumerate(txt1):
        if '태조실록' in i:
            if ('년'and '월' and '일') in i:
                date.append(num)

    for k in range(len(date)-1):
        count =0
        for j in range(date[k],date[k+1]):
            if '제사' in txt1[j]:
                count +=1
        if count >0:
            keywordtxt.append(txt1[date[k]:date[k+1]])
    data.append(trans(keywordtxt,'태조실록'))


In [30]:
df=pd.Series(keywordtxt)
df=df.map(lambda x : re.sub('[^ ㄱ-ㅣ가-힣]+','',str(x)))
df=df.map(lambda x : x.replace("태조실록"," "))
df=df.map(lambda x : x.replace("태조"," "))
df=df.map(lambda x : x.replace("번째기사"," "))
df=df.map(lambda x : x.replace("홍무"," "))

In [31]:
len(df)

169

In [32]:
stopword = ['전하','임금','사람','여러','나라','로써','또한','지금','어찌'
           ]
text=[]
for i in range(0,len(df)):
    t=okt.nouns(str(df[i]))
    temp =[]
    for j in t:
        if j not in stopword:
            if len(j)>1:
                temp.append(j)
    text.append(temp)

In [33]:
dictionary = corpora.Dictionary(text)
corpus = [dictionary.doc2bow(t) for t in text]

In [34]:
NUM_TOPICS = 20

NUM_TOPIC_WORDS = 10

def print_topic_words(model):
    print("\nPrinting topic words.\n")
    
    for topic_id in range(model.num_topics):
        topic_word_probs = model.show_topic(topic_id, NUM_TOPIC_WORDS)
        print("Topic ID: {}".format(topic_id))
        
        for topic_word, prob in topic_word_probs:
            print("\t{}\t{}".format(topic_word, prob))
            
        print("\n")

corpus, dictionary = build_doc_term_mat(text)
model = gensim.models.ldamodel.LdaModel(corpus, num_topics=NUM_TOPICS, id2word=dictionary, alpha="auto", eta="auto")
print_topic_words(model)


Printing topic words.

Topic ID: 0
	제사	0.01136048510670662
	모두	0.0075319488532841206
	친히	0.005329135339707136
	백성	0.005240569822490215
	군사	0.004412010312080383
	하소	0.004359469749033451
	의논	0.003958022687584162
	도평의사사	0.003926136065274477
	고려	0.00373024377040565
	정안군	0.003487273585051298


Topic ID: 1
	모두	0.01130057405680418
	제사	0.010792030021548271
	백성	0.009469310753047466
	왕위	0.006408753804862499
	도평의사사	0.005794602446258068
	절제	0.00521302642300725
	장군	0.005204424727708101
	군사	0.005129637662321329
	제도	0.004528067074716091
	정안군	0.004280388355255127


Topic ID: 2
	제사	0.009210238233208656
	백성	0.007052625995129347
	정안군	0.005807808134704828
	모두	0.005649049300700426
	군사	0.005516336299479008
	생각	0.0034403984900563955
	세자	0.0034242356196045876
	하늘	0.003138734260573983
	하니	0.003106487914919853
	의논	0.0030217948369681835


Topic ID: 3
	제사	0.009798672050237656
	모두	0.008694294840097427
	공신	0.005995399784296751
	중추원	0.005977960303425789
	백성	0.00587446428835392
	서로	0.005184659734368324
	우리	0.0046970

In [35]:
# 제사
pyLDAvis.enable_notebook()
data = pyLDAvis.gensim.prepare(model, corpus, dictionary)
data 

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
17     0.051226  0.000813       1        1  10.508348
12    -0.004004  0.012996       2        1  10.190829
13    -0.015386  0.045583       3        1   9.734067
1      0.038085  0.022619       4        1   9.579885
18     0.034085  0.035700       5        1   9.264231
5      0.026608 -0.006392       6        1   6.805024
0      0.025484  0.000741       7        1   6.578594
7     -0.027572  0.051744       8        1   5.984500
9     -0.075549 -0.034413       9        1   5.527315
11     0.012855 -0.037091      10        1   4.823142
3     -0.029826 -0.014092      11        1   3.876929
6     -0.041273 -0.005348      12        1   3.292140
4     -0.020487 -0.038003      13        1   3.090747
10    -0.053678 -0.012642      14        1   3.059470
15     0.004410  0.058775      15        1   2.374610
14     0.047595 -0.064756      16        1   1.765187
2     -0.011377  0.007276      17        1   1.540280
8      0.043169 -0.026399      18        1   1.488145
16    -0.004870 -0.006299      19        1   0.304347
19     0.000504  0.009186      20        1   0.212211, topic_info=     Term        Freq       Total Category  logprob  loglift
9      제사  216.000000  216.000000  Default  30.0000  30.0000
220    수군   37.000000   37.000000  Default  29.0000  29.0000
1593  정안군   64.000000   64.000000  Default  28.0000  28.0000
50     군사   77.000000   77.000000  Default  27.0000  27.0000
7      절제   59.000000   59.000000  Default  26.0000  26.0000
...   ...         ...         ...      ...      ...      ...
535    다만    0.076003   28.461555  Topic20  -6.0271   0.2298
154    처음    0.074046   28.472271  Topic20  -6.0532   0.2033
163    하니    0.074062   40.225822  Topic20  -6.0530  -0.1420
164    하소    0.074706   44.204697  Topic20  -6.0443  -0.2277
80     백성    0.073504   86.585716  Topic20  -6.0606  -0.9162

[1289 rows x 6 columns], token_table=      Topic      Freq Term
term                      
926      12  0.789997   가마
715       8  0.424348   가묘
1334     11  0.330199   가뭄
3796      4  0.662118   가정
3710      8  0.894129   가족
...     ...       ...  ...
950      12  0.842877  훈도관
3976      8  0.678979   훼철
3190      5  0.572557   흑색
884       1  0.416051   흥덕
1201      9  0.845731   희수

[2561 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[18, 13, 14, 2, 19, 6, 1, 8, 10, 12, 4, 7, 5, 11, 16, 15, 3, 9, 17, 20])